In [1]:
import pandas as pd
import numpy as np
from functools import lru_cache
from numba import njit, prange
import itertools
from scipy import stats
import owlready2
from collections import Counter

ModuleNotFoundError: No module named 'owlready2'

## getting all possible types of values

In [ ]:
import cellxgene_census
import cellxgene_census.experimental.ml as census_ml
import tiledbsoma as soma

census = cellxgene_census.open_soma(census_version = "latest")
experiment = census["census_data"]["homo_sapiens"]
datasets = census["census_info"]["datasets"]

In [8]:
experiment = census["census_data"]['mus_musculus']

In [4]:
batch = ['self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'assay_ontology_term_id', 'dataset_id']
features = ['tissue_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id']

In [5]:
df = census["census_data"]["homo_sapiens"].obs.read(column_names=batch+features, value_filter="is_primary_data == True").concat().to_pandas()

In [9]:
df.shape

(37659244, 9)

In [10]:
df2 = census["census_data"]["homo_sapiens"].obs.read(column_names=batch+features, value_filter="is_primary_data == True").concat().to_pandas()

In [11]:
df2

,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,assay_ontology_term_id,dataset_id,tissue_ontology_term_id,cell_type_ontology_term_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
0,unknown,PATO:0000383,EFO:0009900,f7ec7bd5-04ab-453b-a8a7-c9d14812affb,UBERON:0001238,CL:0000499,HsapDv:0000206,PATO:0000461,True
1,unknown,PATO:0000383,EFO:0009900,f7ec7bd5-04ab-453b-a8a7-c9d14812affb,UBERON:0001238,CL:0000499,HsapDv:0000206,PATO:0000461,True
2,unknown,PATO:0000383,EFO:0009900,f7ec7bd5-04ab-453b-a8a7-c9d14812affb,UBERON:0001238,CL:0000499,HsapDv:0000206,PATO:0000461,True
3,unknown,PATO:0000383,EFO:0009900,f7ec7bd5-04ab-453b-a8a7-c9d14812affb,UBERON:0001238,CL:0000499,HsapDv:0000206,PATO:0000461,True
4,unknown,PATO:0000383,EFO:0009900,f7ec7bd5-04ab-453b-a8a7-c9d14812affb,UBERON:0001238,CL:0000499,HsapDv:0000206,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...
37659239,unknown,PATO:0000383,EFO:0009899,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,UBERON:0002048,CL:0000669,HsapDv:0000145,PATO:0000461,True
37659240,unknown,PATO:0000383,EFO:0009899,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,UBERON:0002048,CL:0000669,HsapDv:0000145,PATO:0000461,True
37659241,unknown,PATO:0000383,EFO:0009899,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,UBERON:0002048,CL:0000669,HsapDv:0000145,PATO:0000461,True
37659242,unknown,PATO:0000383,EFO:0009899,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,UBERON:0002048,CL:0000669,HsapDv:0000145,PATO:0000461,True


In [14]:
df = pd.concat([df,df2])

In [16]:
df.assay_ontology_term_id.unique()

array(['EFO:0009900', 'EFO:0009922', 'EFO:0009899', 'EFO:0700016',
       'EFO:0030004', 'EFO:0011025', 'EFO:0008722', 'EFO:0030003',
       'EFO:0008919', 'EFO:0009901', 'EFO:0008796', 'EFO:0008931',
       'EFO:0700003', 'EFO:0700011', 'EFO:0008780', 'EFO:0030002',
       'EFO:0008953', 'EFO:0010010', 'EFO:0700004', 'EFO:0010550'],
      dtype=object)

In [ ]:
['EFO:0009900', 'EFO:0009922', 'EFO:0009899', 'EFO:0700016',
'EFO:0030004', 'EFO:0011025', 'EFO:0008722', 'EFO:0030003',
'EFO:0008919', 'EFO:0009901', 'EFO:0008796', 'EFO:0008931',
'EFO:0700003', 'EFO:0700011', 'EFO:0008780', 'EFO:0030002',
'EFO:0008953', 'EFO:0010010', 'EFO:0700004', 'EFO:0010550']

In [ ]:
list_of_datasets = {k: val for k, val in Counter(df['datasets']).items() if val >1000}

In [41]:
df = 

KeyboardInterrupt: 

In [8]:
df.shape

(37659244, 9)

In [164]:
from collections import Counter

In [165]:
# less than N cel per dataset?


Counter({'f7c1c579-2dc0-47e2-ba19-8165c5a0e353': 4062980,
         '9f222629-9e39-47d0-b83f-e08d610c7479': 1959503,
         '9dbab10c-118d-496b-966a-67f1763a6b7d': 1462702,
         '6f7fd0f1-a2ed-4ff1-80d3-33dde731cbc3': 1309414,
         '218acb0f-9f2f-4f76-b90b-15a4b7c7f629': 1263676,
         '3faad104-2ab8-4434-816d-474d8d2641db': 1248980,
         'c2876b1b-06d8-4d96-a56b-5304f815b99a': 1149751,
         '56c4912d-2bae-4b64-98f2-af8a84389208': 1092789,
         'b0e547f0-462b-4f81-b31b-5b0a5d96f537': 1058909,
         '1e6a6ef9-7ec9-4c90-bbfb-2ad3c3165fd1': 996759,
         'ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3': 836148,
         '842c6f5d-4a94-4eef-8510-8c792d1124bc': 714331,
         '1a38e762-2465-418f-b81c-6a4bce261c34': 700391,
         '65badd7a-9262-4fd1-9ce2-eb5dc0ca8039': 665955,
         'c7775e88-49bf-4ba2-a03b-93f00447c958': 647366,
         '01ad3cd7-3929-4654-84c0-6db05bd5fd59': 600929,
         '2adb1f8a-a6b1-4909-8ee8-484814e2d4bf': 598266,
         'd4e69e01-3ba

In [166]:
df['group'] = df[features+batch[:-1]].astype(str).agg('_'.join, axis=1)

In [ ]:
for k, val in counts.items():
    print()

In [168]:
# compute weightings
# weight on large dev stage status, cell type tissue, disease, assay
counts = Counter(df['group'])

In [ ]:
# compute the set of datasets to load

## cell line & tissue grouping

In [17]:
CL_BASIC_PERMANENT_URL_OWL = "https://github.com/obophenotype/cell-ontology/releases/latest/download/cl-basic.owl"
UB_BASIC_PERMANENT_URL_OWL = "https://github.com/obophenotype/uberon/releases/latest/download/uberon-basic.owl"
AC_BASIC_PERMANENT_URL_OWL = "https://raw.githubusercontent.com/EBISPOT/hancestro/main/hancestro-base.owl"
AS_BASIC_PERMANENT_URL_OWL = "https://github.com/obophenotype/uberon/releases/latest/download/uberon-basic.owl"
DV_BASIC_PERMANENT_URL_OWL = "http://purl.obolibrary.org/obo/hsapdv.owl"
DI_BASIC_PERMANENT_URL_OWL = 'https://raw.githubusercontent.com/EBISPOT/efo/master/efo-base.owl'


In [155]:
development_iri = "HsapDv_0000206"
val = _ancestors_dv(development_iri)


KeyboardInterrupt



In [103]:
ontology_dn = owlready2.get_ontology("http://purl.obolibrary.org/obo/hsapdv.owl")
ontology_dn.load()

get_ontology("http://purl.obolibrary.org/obo/hsapdv.owl#")

In [144]:

ontology_cl = owlready2.get_ontology(CL_BASIC_PERMANENT_URL_OWL)
ontology_cl.load()
ontology_ub = owlready2.get_ontology(UB_BASIC_PERMANENT_URL_OWL)
ontology_ub.load()
ontology_ac = owlready2.get_ontology(AC_BASIC_PERMANENT_URL_OWL)
ontology_ac.load()
ontology_di = owlready2.get_ontology(DI_BASIC_PERMANENT_URL_OWL)
ontology_di.load()

TypeError: World.get_ontology() missing 1 required positional argument: 'base_iri'

In [152]:
_ancestors_ti("UBERON:0002078")

['Thing',
 'UBERON:0000467',
 'UBERON:0000465',
 'UBERON:0002078',
 'EFO:0000001',
 'EFO:0000635',
 'UBERON:0004535',
 'UBERON:0035554',
 'UBERON:0000064',
 'UBERON:0001062',
 'BFO:0000004',
 'UBERON:0000061',
 'EFO:0001955',
 'EFO:0000786',
 'UBERON:0004120',
 'UBERON:0015212',
 'BFO:0000040',
 'UBERON:0010000',
 'BFO:0000001',
 'BFO:0000002',
 'UBERON:0004151',
 'UBERON:0002081']

In [153]:
@lru_cache(maxsize=None)
def _ancestors_dv(development):
    global ontology_dn
    ancestors = set()
    development_iri = development.replace(":", "_")
    entity = ontology_dn.search_one(iri=f"http://purl.obolibrary.org/obo/{development_iri}")
    for val in entity.ancestors(include_constructs = True, include_self = False):
        try:
            ancestors.add(val.name)
        except AttributeError:
            ancestors.add(val.value.name)
            ancestors |= _ancestors_dv(val.value.name)
    print(len(ancestors))
    return ancestors

In [148]:
@lru_cache(maxsize=None)
def _ancestors_di(disease):
    global ontology_di
    disease_iri = disease.replace(":", "_")
    entity = ontology_di.search_one(iri=f"http://purl.obolibrary.org/obo/{disease_iri}")
    ancestors = (
        [i.name.replace("_", ":") for i in entity.ancestors()]
        if entity
        else [disease]
    )
    return ancestors

@lru_cache(maxsize=None)
def _ancestors_dv(development):
    global ontology_dv
    development_iri = development.replace(":", "_")
    entity = ontology_dv.search_one(iri=f"http://purl.obolibrary.org/obo/{development_iri}")
    ancestors = (
        [i.name.replace("_", ":") for i in entity.ancestors()]
        if entity
        else [development]
    )
    return ancestors

@lru_cache(maxsize=None)
def _ancestors_ac(ancestry):
    global ontology_ac
    ancestry_iri = ancestry.replace(":", "_")
    entity = ontology_ac.search_one(iri=f"http://purl.obolibrary.org/obo/{ancestry_iri}")
    ancestors = (
        [i.name.replace("_", ":") for i in entity.ancestors()]
        if entity
        else [ancestry]
    )
    return ancestors

In [ ]:
@lru_cache(maxsize=None)
def _ancestors_cl(cell_type):
    global ontology_cl
    cell_type_iri = cell_type.replace(":", "_")
    entity = ontology_cl.search_one(iri=f"http://purl.obolibrary.org/obo/{cell_type_iri}")
    ancestors = (
        [i.name.replace("_", ":") for i in entity.ancestors()]
        if entity
        else [cell_type]
    )
    return ancestors

In [151]:
@lru_cache(maxsize=None)
def _ancestors_ti(tissue):
    global ontology_ti
    tissue_iri = tissue.replace(":", "_")
    entity = ontology_ub.search_one(iri=f"http://purl.obolibrary.org/obo/{tissue_iri}")
    ancestors = (
        [i.name.replace("_", ":") for i in entity.ancestors()]
        if entity
        else [tissue]
    )
    return ancestors

In [8]:
import json
import urllib.request

url = "https://raw.githubusercontent.com/chanzuckerberg/single-cell-data-portal/main/frontend/src/components/common/Filter/descendant_mappings/cell_type_descendants.json"

In [17]:
# get all CxG cell types


def get_ancestry_mapping(url, type="cell_type"):
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    tot = set()
    ntot = set()
    for i, j in data.items():
        tot |= set(j)
        ntot |= set([i])
    # for each cell type, get all its ancestors
    ancestors = {}
    for val in tot | ntot:
        if type=="cell_type":
            ancestors[val] = set(_ancestors(val)) - set([val, 'Thing'])
        elif type=="tissue":
            ancestors[val] = set(_ancestors_ti(val)) - set([val, 'Thing'])
        elif type=="ancestry":
            ancestors[val] = set(_ancestors_ac(val)) - set([val, 'Thing'])
        else:
            raise ValueError("type must be 'cell_type' or 'tissue'")
    full_ancestors = set()
    for val in ancestors.values():
        full_ancestors |= set(val)

    # remove the things that are not in CxG
    full_ancestors = full_ancestors & set(ancestors.keys())

    # if a cell type is not an ancestor then it is a leaf
    leafs = tot - full_ancestors
    full_ancestors = full_ancestors - leafs
    # for each ancestor, make a dict of groupings of leafs that predict it
    groupings = {}
    for val in full_ancestors:
        groupings[val] = set()
    for leaf in leafs:
        for ancestor in ancestors[leaf]:
            if ancestor in full_ancestors:
                groupings[ancestor].add(leaf)

    return groupings, full_ancestors, leafs



In [19]:
df.columns

Index(['self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id',
       'assay_ontology_term_id', 'dataset_id', 'tissue_ontology_term_id',
       'cell_type_ontology_term_id', 'development_stage_ontology_term_id',
       'disease_ontology_term_id', 'is_primary_data'],
      dtype='object')

In [63]:
tot

{'HsapDv:0000002',
 'HsapDv:0000015',
 'HsapDv:0000021',
 'HsapDv:0000023',
 'HsapDv:0000024',
 'HsapDv:0000025',
 'HsapDv:0000026',
 'HsapDv:0000027',
 'HsapDv:0000028',
 'HsapDv:0000029',
 'HsapDv:0000030',
 'HsapDv:0000046',
 'HsapDv:0000047',
 'HsapDv:0000048',
 'HsapDv:0000049',
 'HsapDv:0000050',
 'HsapDv:0000051',
 'HsapDv:0000052',
 'HsapDv:0000053',
 'HsapDv:0000054',
 'HsapDv:0000055',
 'HsapDv:0000056',
 'HsapDv:0000057',
 'HsapDv:0000058',
 'HsapDv:0000059',
 'HsapDv:0000060',
 'HsapDv:0000062',
 'HsapDv:0000063',
 'HsapDv:0000068',
 'HsapDv:0000080',
 'HsapDv:0000081',
 'HsapDv:0000082',
 'HsapDv:0000083',
 'HsapDv:0000084',
 'HsapDv:0000085',
 'HsapDv:0000086',
 'HsapDv:0000087',
 'HsapDv:0000088',
 'HsapDv:0000089',
 'HsapDv:0000090',
 'HsapDv:0000091',
 'HsapDv:0000092',
 'HsapDv:0000093',
 'HsapDv:0000094',
 'HsapDv:0000095',
 'HsapDv:0000096',
 'HsapDv:0000097',
 'HsapDv:0000098',
 'HsapDv:0000099',
 'HsapDv:0000100',
 'HsapDv:0000101',
 'HsapDv:0000102',
 'HsapDv:000

In [61]:
ancestors = {}
tot = set(df['development_stage_ontology_term_id'].unique())
for val in tot:
    ancestors[val] = set(_ancestors_dv(val)) - set([val, 'Thing'])
full_ancestors = set()
for val in ancestors.values():
    full_ancestors |= set(val)

# remove the things that are not in CxG
full_ancestors = full_ancestors & set(ancestors.keys())

# if a cell type is not an ancestor then it is a leaf
leafs = tot - full_ancestors
full_ancestors = full_ancestors - leafs
# for each ancestor, make a dict of groupings of leafs that predict it
groupings = {}
for val in full_ancestors:
    groupings[val] = set()
for leaf in leafs:
    for ancestor in ancestors[leaf]:
        if ancestor in full_ancestors:
            groupings[ancestor].add(leaf)

In [62]:
len(leafs), len(full_ancestors), len(groupings)

(172, 0, 0)

In [60]:
groupings, full_ancestors, leafs = get_ancestry_mapping(url, type="cell_type")

In [61]:
len(leafs), len(full_ancestors), len(groupings)

(387, 260, 260)

In [10]:
url = "https://raw.githubusercontent.com/chanzuckerberg/single-cell-data-portal/main/frontend/src/components/common/Filter/descendant_mappings/tissue_descendants.json"

In [11]:
groupings, full_ancestors, leafs = get_ancestry_mapping(data, type="tissue")

In [55]:
len(leafs), len(full_ancestors), len(groupings)

(250, 29, 29)

In [12]:
groupings

{'UBERON:0003528': {'UBERON:0002023',
  'UBERON:0002435',
  'UBERON:0002810',
  'UBERON:0002811',
  'UBERON:0006514',
  'UBERON:0007628',
  'UBERON:0010225',
  'UBERON:0016530',
  'UBERON:0016538',
  'UBERON:0016540'},
 'UBERON:0002081': {'UBERON:0002078', 'UBERON:0002079'},
 'UBERON:0001870': {'UBERON:0002810', 'UBERON:0002811'},
 'UBERON:0001637': {'UBERON:0001621', 'UBERON:0001624', 'UBERON:0005616'},
 'UBERON:0002385': {'UBERON:0001134', 'UBERON:0004648'},
 'UBERON:0003968': {'UBERON:0001542'},
 'UBERON:0000014': {'UBERON:0001416',
  'UBERON:0001471',
  'UBERON:0001511',
  'UBERON:0001868',
  'UBERON:8300000'},
 'UBERON:0002190': {'UBERON:0014455'},
 'UBERON:0001871': {'UBERON:0002808', 'UBERON:0002809'},
 'UBERON:0002021': {'UBERON:0002807'},
 'UBERON:0002378': {'UBERON:0002382'},
 'UBERON:0002082': {'UBERON:0002080', 'UBERON:0002084'},
 'UBERON:0001630': {'UBERON:0001103',
  'UBERON:0001388',
  'UBERON:0002382',
  'UBERON:0008612'},
 'UBERON:0001085': {'UBERON:0001416', 'UBERON:0

In [58]:
leafs

{'UBERON:0000002',
 'UBERON:0000004',
 'UBERON:0000006',
 'UBERON:0000016',
 'UBERON:0000017',
 'UBERON:0000053',
 'UBERON:0000056',
 'UBERON:0000057',
 'UBERON:0000059',
 'UBERON:0000074',
 'UBERON:0000080',
 'UBERON:0000088 (organoid)',
 'UBERON:0000160',
 'UBERON:0000310 (organoid)',
 'UBERON:0000328',
 'UBERON:0000362',
 'UBERON:0000400',
 'UBERON:0000411',
 'UBERON:0000416',
 'UBERON:0000451',
 'UBERON:0000453',
 'UBERON:0000473',
 'UBERON:0000926',
 'UBERON:0000945',
 'UBERON:0000947',
 'UBERON:0000948',
 'UBERON:0000955',
 'UBERON:0000956',
 'UBERON:0000964',
 'UBERON:0000965',
 'UBERON:0000966',
 'UBERON:0000966 (organoid)',
 'UBERON:0000970',
 'UBERON:0000977',
 'UBERON:0000988',
 'UBERON:0000995',
 'UBERON:0000995 (organoid)',
 'UBERON:0001003',
 'UBERON:0001005',
 'UBERON:0001043',
 'UBERON:0001046',
 'UBERON:0001052',
 'UBERON:0001103',
 'UBERON:0001117',
 'UBERON:0001134',
 'UBERON:0001153',
 'UBERON:0001154',
 'UBERON:0001155',
 'UBERON:0001156',
 'UBERON:0001157',
 'UBER

In [ ]:
main_groups = {
    "adipose tissue": "",
    "bladder organ": "",
    "blood": "",
    "bone marrow": "",
    "brain": "",
    "breast": "",
    "esophagus": "",
    "eye": "",
    "embryo": "",
    "fallopian tube": "",
    "gall bladder": "",
    "heart": "",
    "intestine": "",
    "kidney": "",
    "liver": "",
    "lung": "",
    "lymph node": "",
    "musculature of body": "",
    "nose": "",
    "ovary": "",
    "pancreas": "",
    "placenta": "",
    "skin of body": "",
    "spinal cord": "",
    "spleen": "",
    "stomach": "",
    "thymus": "",
    "thyroid gland": "",
    "tongue": "",
    "uterus": "",
}

In [57]:
groupings

{'UBERON:0001902': {'UBERON:0000400', 'UBERON:0008345'},
 'UBERON:0001872': {'UBERON:0002802', 'UBERON:0002803'},
 'UBERON:0002378': {'UBERON:0002382'},
 'UBERON:0003661': {'UBERON:0001388'},
 'UBERON:0001013': {'UBERON:0001348',
  'UBERON:0003428',
  'UBERON:0005406',
  'UBERON:0014454',
  'UBERON:0014455',
  'UBERON:0015143'},
 'UBERON:0000178': {'UBERON:0012168', 'UBERON:0013756'},
 'UBERON:0000991': {'UBERON:0000473', 'UBERON:0002118', 'UBERON:0002119'},
 'UBERON:0002420': {'UBERON:0002023'},
 'UBERON:0000014': {'UBERON:0001416',
  'UBERON:0001471',
  'UBERON:0001511',
  'UBERON:0001868',
  'UBERON:8300000'},
 'UBERON:0002021': {'UBERON:0002807'},
 'UBERON:0007644': {'UBERON:0039167'},
 'UBERON:0000397': {'UBERON:0005636'},
 'UBERON:0001085': {'UBERON:0001416', 'UBERON:0001868'},
 'UBERON:0001870': {'UBERON:0002810', 'UBERON:0002811'},
 'UBERON:0002082': {'UBERON:0002080', 'UBERON:0002084'},
 'UBERON:0000992': {'UBERON:0002118', 'UBERON:0002119'},
 'UBERON:0002190': {'UBERON:001445

In [56]:
groupings['UBERON:0001723']

KeyError: 'UBERON:0001723'

In [40]:
response = urllib.request.urlopen(url)
data = json.loads(response.read())

## getting GRNs

In [2]:
import decoupler as dc
dc.get_collectri(organism='human', split_complexes=False)

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

,source,target,weight,PMID
0,MYC,TERT,1,10022128;10491298;10606235;10637317;10723141;1...
1,SPI1,BGLAP,1,10022617
2,SMAD3,JUN,1,10022869;12374795
3,SMAD4,JUN,1,10022869;12374795
4,STAT5A,IL2,1,10022878;11435608;17182565;17911616;22854263;2...
...,...,...,...,...
43173,NFKB,hsa-miR-143-3p,1,19472311
43174,AP1,hsa-miR-206,1,19721712
43175,NFKB,hsa-miR-21-5p,1,20813833;22387281
43176,NFKB,hsa-miR-224-5p,1,23474441;23988648


In [5]:
import omnipath as op
op.interactions.CollecTRI.get(genesymbols=True, organism='human', loops=True)

,source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,curation_effort,references,sources,n_sources,n_primary_sources,n_references,references_stripped
0,P01106,O14746,MYC,TERT,False,True,False,True,True,False,82,CollecTRI:10022128;CollecTRI:10491298;CollecTR...,CollecTRI;DoRothEA-A_CollecTRI;ExTRI_CollecTRI...,8,1,74,10022128;10491298;10606235;10637317;10723141;1...
1,P17947,P02818,SPI1,BGLAP,False,True,False,True,True,False,3,CollecTRI:10022617,CollecTRI;ExTRI_CollecTRI,2,1,1,10022617
2,COMPLEX:P15407_P17275,P05412,FOSL1_JUNB,JUN,False,True,False,True,True,False,53,CollecTRI:10022869;CollecTRI:10037172;CollecTR...,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecT...,4,1,49,10022869;10037172;10208431;10366004;11281649;1...
3,COMPLEX:P01100_P05412,P05412,FOS_JUN,JUN,False,True,False,True,True,False,53,CollecTRI:10022869;CollecTRI:10037172;CollecTR...,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecT...,4,1,49,10022869;10037172;10208431;10366004;11281649;1...
4,COMPLEX:P01100_P17275,P05412,FOS_JUNB,JUN,False,True,False,True,True,False,53,CollecTRI:10022869;CollecTRI:10037172;CollecTR...,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecT...,4,1,49,10022869;10037172;10208431;10366004;11281649;1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64953,Q01196,Q13094,RUNX1,LCP2,False,True,False,True,True,False,3,CollecTRI:20019798,CollecTRI;DoRothEA-A_CollecTRI,2,1,1,20019798
64954,Q01196,Q6MZQ0,RUNX1,PRR5L,False,True,False,True,True,False,3,CollecTRI:20019798,CollecTRI;DoRothEA-A_CollecTRI,2,1,1,20019798
64955,Q15672,P08151,TWIST1,GLI1,False,True,False,True,True,False,3,CollecTRI:11948912,CollecTRI;DoRothEA-A_CollecTRI,2,1,1,11948912
64956,P22415,Q5SRE5,USF1,NUP188,False,True,False,True,True,False,3,CollecTRI:22951020,CollecTRI;DoRothEA-A_CollecTRI,2,1,1,22951020


In [2]:
import os
import urllib.request
import pandas as pd

def pd_load_cached(url, loc='/tmp/', cache=True, **kwargs):
    # Check if the file exists, if not, download it
    loc+=url.split('/')[-1]
    if not os.path.isfile(loc) or not cache:
        urllib.request.urlretrieve(url, loc)
    # Load the data from the file
    return pd.read_csv(loc, **kwargs)
url = 'https://cdn.netbiol.org/tflink/download_files/TFLink_Homo_sapiens_interactions_All_simpleFormat_v1.0.tsv.gz'

data = pd_load_cached(url,  sep='\t')


In [10]:
data.rename({"Name.TF": 'regulator', 'Name.Target': "target"})

,UniprotID.TF,UniprotID.Target,NCBI.GeneID.TF,NCBI.GeneID.Target,Name.TF,Name.Target,Detection.method,PubmedID,Organism,Source.database,Small-scale.evidence,TF.TFLink.ortho,TF.nonTFLink.ortho,Target.TFLink.ortho,Target.nonTFLink.ortho
0,Q9H9S0,O94907,79923,22943,NANOG,DKK1,chromatin immunoprecipitation assay;inferred b...,19148141;29087512;29126285;27924024,Homo sapiens,GTRD;ReMap;TRRUST,Yes,-,-,Dr:Q9PWH3;Dr:F1RBK0;Mm:O54908,Rn:D3Z9J1
1,P37231,P10826,5468,5915,PPARG,RARB,chromatin immunoprecipitation assay;inferred b...,17202159;12839938;29087512;27924024,Homo sapiens,GTRD;TRED;TRRUST,Yes,-,-,Mm:P22605;Rn:D3ZFD9,-
2,P10242,P08047,4602,6667,MYB,SP1,chromatin immunoprecipitation assay;inferred b...,29126285;27924024;17202159,Homo sapiens,GTRD;ReMap;TRED,Yes,Dr:F1QP24;Rn:A0A0G2K2A4,Mm:A0A087WPA7,Dr:F1QW97;Rn:Q01714,Mm:G3X8Q0
3,P31269,Q02363,3205,3398,HOXA9,ID2,inferred by curator,29087512;20565746,Homo sapiens,TRRUST,Yes,Mm:P09631,Rn:D3ZSU5,Dr:Q7SZQ2;Mm:P41136;Rn:P41137,-
4,P03372,P17275,2099,3726,ESR1,JUNB,chromatin immunoprecipitation assay;inferred b...,29126285;18971253;27924024;11477071;17202159;2...,Homo sapiens,GTRD;PAZAR;ReMap;TRED;TRRUST,Yes,Mm:P19785,Rn:A0A0G2K0D4,Dr:A8E7N0;Mm:P09450;Rn:A0A0G2JSY8,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739352,Q8TF68,Q9HBW0,171017,9170,ZNF384,LPAR2,chromatin immunoprecipitation assay,27924024,Homo sapiens,GTRD,No,Rn:G3V9M5,Dr:I3ITQ6;Mm:E9Q1A5,Dr:F1Q734;Mm:Q6P290;Rn:A0A0G2K505,-
6739353,O96006,P08670,9189,7431,ZBED1,VIM,chromatin immunoprecipitation assay,27924024,Homo sapiens,GTRD,No,-,-,Mm:P20152;Rn:G3V8C3,Dr:F1QAM8;Dr:F1QT60
6739354,Q8IYM9,Q13286,10346,1201,TRIM22,CLN3,chromatin immunoprecipitation assay,29126285;27924024,Homo sapiens,GTRD;ReMap,No,-,-,-,-
6739355,Q9NZC4,Q8N883,26298,80110,EHF,ZNF614,chromatin immunoprecipitation assay,27924024,Homo sapiens,GTRD,No,-,-,-,-


In [3]:
url = 'http://bioinfo.life.hust.edu.cn/static/hTFtarget/file_download/tf-target-infomation.txt'

data = pd_load_cached(url,  sep='\t')

In [4]:
data.

,TF,target,tissue
0,AEBP2,TMEM53,colon
1,AEBP2,C1orf228,colon
2,AEBP2,FBXO31,colon
3,AEBP2,ADAMTSL5,colon
4,AEBP2,CTB-25B13.9,colon
...,...,...,...
1342124,CCDC101,CLN3,unclear
1342125,CCDC101,RP11-666O2.1,unclear
1342126,CCDC101,RP11-666O2.2,unclear
1342127,CCDC101,RP11-666O2.4,unclear


## working with ESM2 embeddings

In [ ]:
! esm-extract esm2_t33_650M_UR50D data/temp/test_esm.fasta data/temp/ --include mean

In [25]:
import io
import os
from biomart import BiomartServer

def _fetchFromServer(ensemble_server, attributes):
    server = BiomartServer(ensemble_server, verbose=True)
    ensmbl = server.datasets["hsapiens_gene_ensembl"]
    print(attributes)
    res = pd.read_csv(
        io.StringIO(
            ensmbl.search({"attributes": attributes}, header=1).content.decode()
        ),
        sep="\t",
    )
    return res

def createFoldersFor(filepath):
    """
    will recursively create folders if needed until having all the folders required to save the file in this filepath
    """
    prevval = ""
    for val in os.path.expanduser(filepath).split("/")[:-1]:
        prevval += val + "/"
        if not os.path.exists(prevval):
            os.mkdir(prevval)

def getBiomartTable(
    ensemble_server="http://feb2023.archive.ensembl.org/biomart",
    useCache=False,
    cache_folder="/tmp/biomart/",
    attributes=[],
    bypass_attributes=False,
):
    """generate a genelist dataframe from ensembl's biomart

    Args:
        ensemble_server ([type], optional): [description]. Defaults to ENSEMBL_SERVER_V.
        useCache (bool, optional): [description]. Defaults to False.
        cache_folder ([type], optional): [description]. Defaults to CACHE_PATH.

    Raises:
        ValueError: [description]

    Returns:
        [type]: [description]
    """
    attr = (
        [
            "ensembl_gene_id",
            "hgnc_symbol",
            "gene_biotype",
            "entrezgene_id",
        ]
        if not bypass_attributes
        else []
    )
    assert cache_folder[-1] == "/"

    cache_folder = os.path.expanduser(cache_folder)
    createFoldersFor(cache_folder)
    cachefile = os.path.join(cache_folder, ".biomart.csv")
    if useCache & os.path.isfile(cachefile):
        print("fetching gene names from biomart cache")
        res = pd.read_csv(cachefile)
    else:
        print("downloading gene names from biomart")

        res = _fetchFromServer(ensemble_server, attr + attributes)
        res.to_csv(cachefile, index=False)

    res.columns = attr + attributes
    if type(res) is not type(pd.DataFrame()):
        raise ValueError("should be a dataframe")
    res = res[~(res["ensembl_gene_id"].isna() & res["hgnc_symbol"].isna())]
    res.loc[res[res.hgnc_symbol.isna()].index, "hgnc_symbol"] = res[
        res.hgnc_symbol.isna()
    ]["ensembl_gene_id"]

    return res

In [ ]:
biomart

In [42]:
biomart

,ensembl_gene_id,hgnc_symbol,gene_biotype,entrezgene_id,ensembl_transcript_id,protein_id,ensembl_peptide_id
0,ENSG00000198888,MT-ND1,protein_coding,4535.0,ENST00000361390,CAA24026,ENSP00000354687
1,ENSG00000198888,MT-ND1,protein_coding,4535.0,ENST00000361390,AAB58943,ENSP00000354687
2,ENSG00000198888,MT-ND1,protein_coding,4535.0,ENST00000361390,BAA07290,ENSP00000354687
3,ENSG00000198888,MT-ND1,protein_coding,4535.0,ENST00000361390,AAP89036,ENSP00000354687
4,ENSG00000198888,MT-ND1,protein_coding,4535.0,ENST00000361390,AAP89049,ENSP00000354687
...,...,...,...,...,...,...,...
901591,ENSG00000162437,RAVER2,protein_coding,55225.0,ENST00000418058,NaN,ENSP00000397069
901592,ENSG00000122432,SPATA1,protein_coding,100505741.0,ENST00000697276,NaN,ENSP00000514413
901593,ENSG00000122432,SPATA1,protein_coding,100505741.0,ENST00000699524,NaN,ENSP00000514414
901594,ENSG00000122432,SPATA1,protein_coding,100505741.0,ENST00000697277,NaN,ENSP00000514416


In [38]:
biomart = getBiomartTable(attributes=               
    ["ensembl_transcript_id",
        "protein_id",
        "ensembl_peptide_id",
        #"peptide",
    ], bypass_attributes=False)

downloading gene names from biomart
[BiomartServer:'http://feb2023.archive.ensembl.org/biomart/martservice'] is alive.
[BiomartServer:'http://feb2023.archive.ensembl.org/biomart/martservice'] Fetching datasets
[BiomartServer:'http://feb2023.archive.ensembl.org/biomart/martservice'] Fetching databases
[BiomartDatabase:'Ensembl Genes 109'] Fetching datasets
[BiomartDatabase:'Mouse strains 109'] Fetching datasets
[BiomartDatabase:'Sequence'] Fetching datasets
[BiomartDatabase:'Ontology'] Fetching datasets
[BiomartDatabase:'Genomic features 109'] Fetching datasets
[BiomartDatabase:'Ensembl Variation 109'] Fetching datasets
[BiomartDatabase:'Ensembl Regulation 109'] Fetching datasets
['ensembl_gene_id', 'hgnc_symbol', 'gene_biotype', 'entrezgene_id', 'ensembl_transcript_id', 'protein_id', 'ensembl_peptide_id']
[BiomartDataset:'hsapiens_gene_ensembl'] Searching using following params:
{'attributes': ['ensembl_gene_id',
                'hgnc_symbol',
                'gene_biotype',
          

In [12]:
import os
import glob
import torch

tensor_list = []
for file in glob.glob("../../data/temp/*.pt"):
    tensor = torch.load(file)['mean_representations'][33]
    tensor_list.append(tensor)

concatenated_tensor = torch.cat(tensor_list, dim=0)


{33: tensor([-0.0224,  0.0423, -0.0291,  ..., -0.1396, -0.0235, -0.0449])}